In [1]:
import numpy as np
import heapq

# 定义地图数据（示例）
map_data = np.zeros((50, 50))  # 假设地图大小为50x50
map_data[30:40, 20:30] = 1  # 设置障碍物

# 定义起始点和目标点
start = (43, 32)
target = (40, 39)

# 定义寻路函数
def find_path(start, target, map_data):
    # 定义辅助函数
    def heuristic(a, b):
        return abs(b[0] - a[0]) + abs(b[1] - a[1])  # 曼哈顿距离作为启发式函数

    # 定义节点类
    class Node:
        def __init__(self, position, parent=None):
            self.position = position
            self.parent = parent
            self.g = 0
            self.h = 0
            self.f = 0

        def __lt__(self, other):
            return self.f < other.f

    # 初始化起始节点和目标节点
    start_node = Node(start)
    target_node = Node(target)

    # 初始化开放列表和关闭列表
    open_list = []
    closed_list = set()

    # 将起始节点添加到开放列表
    heapq.heappush(open_list, start_node)

    # 开始寻路
    while open_list:
        # 从开放列表中取出f值最小的节点
        current_node = heapq.heappop(open_list)

        # 将当前节点添加到关闭列表
        closed_list.add(current_node.position)

        # 判断是否到达目标节点
        if current_node.position == target_node.position:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]  # 返回反转后的路径

        # 遍历当前节点周围的邻居节点
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            neighbor_position = (current_node.position[0] + dx, current_node.position[1] + dy)

            # 忽略超出地图边界的节点和障碍物节点
            if (
                neighbor_position[0] < 0
                or neighbor_position[0] >= map_data.shape[0]
                or neighbor_position[1] < 0
                or neighbor_position[1] >= map_data.shape[1]
                or map_data[neighbor_position] == 1
            ):
                continue

            # 创建邻居节点
            neighbor_node = Node(neighbor_position, current_node)

            # 如果邻居节点已在关闭列表中，则忽略
            if neighbor_position in closed_list:
                continue

            # 计算邻居节点的g值、h值和f值
            neighbor_node.g = current_node.g + 1
            neighbor_node.h = heuristic(neighbor_node.position, target_node.position)
            neighbor_node.f = neighbor_node.g + neighbor_node.h

            # 将邻居节点添加到开放列表
            heapq.heappush(open_list, neighbor_node)

    # 如果开放列表为空且没有找到路径，则表示无法到达目标节点
    return None

# 调用寻路函数并打印路径
path = find_path(start, target, map_data)
if path:
    print("找到路径：", path)
else:
    print("无法到达目标点")


找到路径： [(43, 32), (42, 32), (42, 33), (42, 34), (42, 35), (42, 36), (42, 37), (41, 37), (41, 38), (40, 38), (40, 39)]


In [2]:
import time

# 初始化角色位置
player_position = (0, 0)

# 定义移动速度和持续时间
movement_speed = 1  # 每秒移动的格数
button_hold_time = 1  # 按钮按住的持续时间（秒）

# 定义移动函数
def move_player(dx, dy):
    global player_position
    x, y = player_position
    player_position = (x + dx, y + dy)
    print(f"角色移动到：{player_position}")

# 模拟用户输入
def simulate_user_input():
    # 假设用户按住下键1秒钟
    move_duration = 1  # 按键按住的持续时间（秒）
    move_distance = movement_speed * move_duration  # 移动的格数
    move_player(0, -move_distance)

# 主循环
def main_loop():
    while True:
        # 监听用户输入
        # 这里使用 time.sleep 模拟主循环的延时
        # 实际情况下，你可能需要使用适当的输入函数来监听用户输入
        simulate_user_input()
        time.sleep(1)  # 主循环延时

# 运行主循环
main_loop()


角色移动到：(0, -1)
角色移动到：(0, -2)
角色移动到：(0, -3)
角色移动到：(0, -4)
角色移动到：(0, -5)
角色移动到：(0, -6)
角色移动到：(0, -7)
角色移动到：(0, -8)


KeyboardInterrupt: 

In [3]:
import time

# 初始化角色位置和目标位置
player_position = (0, 0)
target_position = (0, 0)

# 定义移动速度
movement_speed = 1  # 每秒移动的格数

# 模拟用户输入，更新目标位置
def update_target_position(x, y):
    global target_position
    target_position = (x, y)
    print(f"目标位置更新为：{target_position}")

# 计算两点之间的距离
def calculate_distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

# 移动函数
def move_player(dx, dy):
    global player_position
    x, y = player_position
    player_position = (x + dx, y + dy)
    print(f"角色移动到：{player_position}")

# 主循环
def main_loop():
    while True:
        # 获取当前位置和目标位置
        current_position = player_position

        # 检查是否到达目标位置
        if current_position == target_position:
            print("已经到达目标位置")
            break

        # 计算当前位置与目标位置之间的距离
        distance = calculate_distance(current_position, target_position)

        # 计算每次移动的步数
        move_distance = min(movement_speed, distance)

        # 计算移动的方向向量
        dx = int((target_position[0] - current_position[0]) * move_distance / distance)
        dy = int((target_position[1] - current_position[1]) * move_distance / distance)

        # 移动角色
        move_player(dx, dy)

        # 延时，模拟主循环的时间间隔
        time.sleep(1)

# 示例：更新目标位置
update_target_position(5, 5)

# 运行主循环
main_loop()


目标位置更新为：(5, 5)
角色移动到：(0, 0)
角色移动到：(0, 0)
角色移动到：(0, 0)
角色移动到：(0, 0)
角色移动到：(0, 0)


KeyboardInterrupt: 